# CHARGED Dataset Visualization - Six Cities Analysis

This notebook provides comprehensive visualizations and analysis of EV charging data across six global cities:
- Amsterdam (AMS)
- Johannesburg (JHB)
- Los Angeles (LOA)
- Melbourne (MEL)
- São Paulo (SPO)
- Shenzhen (SZH)

## Contents:
1. Interactive Time-Based Map Visualization
2. Data Science Analysis & Clustering
3. Temporal Patterns & Insights

In [ ]:
%pip install seaborn

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# For interactive maps
import folium
from folium import plugins
from folium.plugins import HeatMapWithTime, MarkerCluster

# For clustering
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("Libraries imported successfully!")

## 1. Data Loading

Load data for all six cities including:
- Sites (locations with coordinates)
- Volume (charging session counts over time)
- Duration (charging session durations)

In [ ]:
# Define cities
CITIES = ['AMS', 'JHB', 'LOA', 'MEL', 'SPO', 'SZH']
CITY_NAMES = {
    'AMS': 'Amsterdam',
    'JHB': 'Johannesburg',
    'LOA': 'Los Angeles',
    'MEL': 'Melbourne',
    'SPO': 'São Paulo',
    'SZH': 'Shenzhen'
}

# Base data directory
DATA_DIR = Path('/home/mengyuwsl/CHARGED/data')

# Load data for all cities
sites_data = {}
volume_data = {}
duration_data = {}

for city in CITIES:
    try:
        # Load sites
        sites_path = DATA_DIR / city / 'sites.csv'
        sites_data[city] = pd.read_csv(sites_path)
        print(f"{CITY_NAMES[city]}: {len(sites_data[city])} charging sites")
        
        # Load volume data
        volume_path = DATA_DIR / city / 'volume.csv'
        volume_df = pd.read_csv(volume_path)
        volume_df.rename(columns={volume_df.columns[0]: 'timestamp'}, inplace=True)
        volume_df['timestamp'] = pd.to_datetime(volume_df['timestamp'])
        volume_data[city] = volume_df
        
        # Load duration data
        duration_path = DATA_DIR / city / 'duration.csv'
        duration_df = pd.read_csv(duration_path)
        duration_df.rename(columns={duration_df.columns[0]: 'timestamp'}, inplace=True)
        duration_df['timestamp'] = pd.to_datetime(duration_df['timestamp'])
        duration_data[city] = duration_df
        
    except Exception as e:
        print(f"Error loading {city}: {e}")

print(f"\nSuccessfully loaded data for {len(sites_data)} cities")

## 2. Interactive Time-Based Map Visualization

Create an interactive map with a time slider showing charging activity evolution across all cities.

In [ ]:
# Prepare data for time-based heatmap
def prepare_heatmap_data(city_code, sample_hours=24*7):  # Sample one week of data
    """
    Prepare heatmap data for a specific city
    Returns list of [lat, lon, weight] for each time step
    """
    sites = sites_data[city_code]
    volume = volume_data[city_code]
    
    # Sample data to avoid overwhelming the visualization
    volume_sample = volume.iloc[:sample_hours]
    
    heatmap_data = []
    time_index = []
    
    for idx, row in volume_sample.iterrows():
        time_index.append(row['timestamp'].strftime('%Y-%m-%d %H:%M'))
        timestep_data = []
        
        # Get volume for each site at this time
        for site_id in sites['site'].values:
            if str(site_id) in volume_sample.columns:
                site_info = sites[sites['site'] == site_id].iloc[0]
                lat = site_info['latitude']
                lon = site_info['longitude']
                weight = row[str(site_id)]
                
                # Only add if there's activity
                if weight > 0:
                    timestep_data.append([lat, lon, float(weight)])
        
        heatmap_data.append(timestep_data)
    
    return heatmap_data, time_index

print("Helper function defined")

In [ ]:
# Create individual city maps with time slider
def create_city_time_map(city_code, sample_hours=24*7):
    """
    Create an interactive map with time slider for a specific city
    """
    sites = sites_data[city_code]
    
    # Calculate center of map
    center_lat = sites['latitude'].mean()
    center_lon = sites['longitude'].mean()
    
    # Create base map
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=11,
        tiles='OpenStreetMap'
    )
    
    # Add site markers
    marker_cluster = MarkerCluster(name='Charging Sites').add_to(m)
    
    for _, site in sites.iterrows():
        folium.CircleMarker(
            location=[site['latitude'], site['longitude']],
            radius=5,
            popup=f"Site {site['site']}<br>Chargers: {site['charger_num']}<br>Total Volume: {site['total_volume']:.0f}",
            color='blue',
            fill=True,
            fillColor='blue',
            fillOpacity=0.6
        ).add_to(marker_cluster)
    
    # Prepare heatmap data
    print(f"Preparing heatmap data for {CITY_NAMES[city_code]}...")
    heatmap_data, time_index = prepare_heatmap_data(city_code, sample_hours)
    
    # Add time-based heatmap
    HeatMapWithTime(
        heatmap_data,
        index=time_index,
        auto_play=True,
        max_opacity=0.8,
        radius=15,
        gradient={0.0: 'blue', 0.5: 'lime', 1.0: 'red'},
        name='Charging Activity Over Time'
    ).add_to(m)
    
    # Add layer control
    folium.LayerControl().add_to(m)
    
    # Add title
    title_html = f'''
    <div style="position: fixed; 
                top: 10px; left: 50px; width: 400px; height: 60px; 
                background-color: white; border:2px solid grey; z-index:9999; 
                font-size:16px; padding: 10px">
        <h4 style="margin:0;">{CITY_NAMES[city_code]} - EV Charging Activity</h4>
        <p style="margin:5px 0 0 0; font-size:12px;">Time-based heatmap showing charging demand evolution</p>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(title_html))
    
    return m

print("Map creation function defined")

In [ ]:
# Create and save maps for each city
# Note: Creating maps for all cities - you can view them individually

for city in CITIES:
    print(f"\nCreating interactive map for {CITY_NAMES[city]}...")
    city_map = create_city_time_map(city, sample_hours=24*7)  # One week of data
    
    # Save map
    output_path = f'{city.lower()}_charging_time_map.html'
    city_map.save(output_path)
    print(f"Map saved to: {output_path}")

print("\n✓ All city maps created successfully!")
print("\nYou can open the HTML files in your browser to interact with the time slider.")

## 3. Combined Multi-City Overview Map

Create a global view showing all cities on one map.

In [ ]:
# Create a world map with all cities
world_map = folium.Map(
    location=[20, 0],  # Center on world
    zoom_start=2,
    tiles='OpenStreetMap'
)

# Color scheme for cities
city_colors = {
    'AMS': 'red',
    'JHB': 'blue', 
    'LOA': 'green',
    'MEL': 'purple',
    'SPO': 'orange',
    'SZH': 'darkred'
}

# Add each city's sites
for city in CITIES:
    sites = sites_data[city]
    feature_group = folium.FeatureGroup(name=CITY_NAMES[city])
    
    for _, site in sites.iterrows():
        folium.CircleMarker(
            location=[site['latitude'], site['longitude']],
            radius=3,
            popup=f"{CITY_NAMES[city]}<br>Site {site['site']}<br>Volume: {site['total_volume']:.0f}",
            color=city_colors[city],
            fill=True,
            fillColor=city_colors[city],
            fillOpacity=0.7
        ).add_to(feature_group)
    
    feature_group.add_to(world_map)

# Add layer control
folium.LayerControl().add_to(world_map)

# Save world map
world_map.save('all_cities_overview_map.html')
print("✓ Global overview map created: all_cities_overview_map.html")

# Display in notebook
world_map

## 4. Data Science Analysis - Basic Statistics

Explore basic statistics and patterns in the charging data.

In [ ]:
# Calculate summary statistics for each city
summary_stats = []

for city in CITIES:
    sites = sites_data[city]
    volume = volume_data[city]
    duration = duration_data[city]
    
    # Calculate statistics
    total_sites = len(sites)
    total_chargers = sites['charger_num'].sum()
    total_sessions = sites['total_volume'].sum()
    avg_duration = sites['total_duration'].mean()
    avg_power = sites['avg_power'].mean()
    
    # Time range
    time_range = f"{volume['timestamp'].min().date()} to {volume['timestamp'].max().date()}"
    days_covered = (volume['timestamp'].max() - volume['timestamp'].min()).days
    
    summary_stats.append({
        'City': CITY_NAMES[city],
        'Code': city,
        'Sites': total_sites,
        'Chargers': total_chargers,
        'Total Sessions': f"{total_sessions:,.0f}",
        'Avg Duration (min)': f"{avg_duration:.1f}",
        'Avg Power (kW)': f"{avg_power:.1f}",
        'Days Covered': days_covered,
        'Time Range': time_range
    })

summary_df = pd.DataFrame(summary_stats)
print("\n📊 CHARGING INFRASTRUCTURE SUMMARY\n")
print(summary_df.to_string(index=False))

# Store for later use
summary_df.to_csv('city_summary_statistics.csv', index=False)
print("\n✓ Summary saved to: city_summary_statistics.csv")

In [ ]:
# Visualize city comparisons
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Charging Infrastructure Comparison Across Cities', fontsize=16, fontweight='bold')

# Extract numeric values for plotting
summary_for_plot = []
for city in CITIES:
    sites = sites_data[city]
    summary_for_plot.append({
        'City': CITY_NAMES[city],
        'Sites': len(sites),
        'Chargers': sites['charger_num'].sum(),
        'Total Sessions': sites['total_volume'].sum(),
        'Avg Duration': sites['total_duration'].mean(),
        'Avg Power': sites['avg_power'].mean(),
        'Avg Area': sites['area'].mean()
    })
plot_df = pd.DataFrame(summary_for_plot)

# 1. Number of sites
axes[0, 0].bar(plot_df['City'], plot_df['Sites'], color=sns.color_palette('husl', 6))
axes[0, 0].set_title('Number of Charging Sites', fontweight='bold')
axes[0, 0].set_ylabel('Count')
axes[0, 0].tick_params(axis='x', rotation=45)

# 2. Number of chargers
axes[0, 1].bar(plot_df['City'], plot_df['Chargers'], color=sns.color_palette('husl', 6))
axes[0, 1].set_title('Total Number of Chargers', fontweight='bold')
axes[0, 1].set_ylabel('Count')
axes[0, 1].tick_params(axis='x', rotation=45)

# 3. Total charging sessions
axes[0, 2].bar(plot_df['City'], plot_df['Total Sessions'], color=sns.color_palette('husl', 6))
axes[0, 2].set_title('Total Charging Sessions', fontweight='bold')
axes[0, 2].set_ylabel('Sessions')
axes[0, 2].tick_params(axis='x', rotation=45)
axes[0, 2].ticklabel_format(style='plain', axis='y')

# 4. Average session duration
axes[1, 0].bar(plot_df['City'], plot_df['Avg Duration'], color=sns.color_palette('husl', 6))
axes[1, 0].set_title('Average Session Duration', fontweight='bold')
axes[1, 0].set_ylabel('Minutes')
axes[1, 0].tick_params(axis='x', rotation=45)

# 5. Average power
axes[1, 1].bar(plot_df['City'], plot_df['Avg Power'], color=sns.color_palette('husl', 6))
axes[1, 1].set_title('Average Charging Power', fontweight='bold')
axes[1, 1].set_ylabel('kW')
axes[1, 1].tick_params(axis='x', rotation=45)

# 6. Average site area
axes[1, 2].bar(plot_df['City'], plot_df['Avg Area'], color=sns.color_palette('husl', 6))
axes[1, 2].set_title('Average Site Area', fontweight='bold')
axes[1, 2].set_ylabel('Square meters')
axes[1, 2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('city_comparison_overview.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Comparison charts saved to: city_comparison_overview.png")

## 5. Temporal Pattern Analysis

Analyze how charging demand varies over time for each city.

In [ ]:
# Analyze hourly patterns
fig, axes = plt.subplots(2, 3, figsize=(20, 10))
fig.suptitle('Hourly Charging Demand Patterns by City', fontsize=16, fontweight='bold')

for idx, city in enumerate(CITIES):
    row = idx // 3
    col = idx % 3
    ax = axes[row, col]
    
    volume = volume_data[city].copy()
    
    # Extract hour and calculate average demand
    volume['hour'] = volume['timestamp'].dt.hour
    
    # Sum across all sites for each hour
    site_cols = [col for col in volume.columns if col not in ['timestamp', 'hour']]
    volume['total_demand'] = volume[site_cols].sum(axis=1)
    
    # Average by hour of day
    hourly_avg = volume.groupby('hour')['total_demand'].mean()
    
    # Plot
    ax.plot(hourly_avg.index, hourly_avg.values, marker='o', linewidth=2, markersize=6)
    ax.fill_between(hourly_avg.index, hourly_avg.values, alpha=0.3)
    ax.set_title(CITY_NAMES[city], fontweight='bold')
    ax.set_xlabel('Hour of Day')
    ax.set_ylabel('Average Charging Sessions')
    ax.grid(True, alpha=0.3)
    ax.set_xticks(range(0, 24, 3))

plt.tight_layout()
plt.savefig('hourly_demand_patterns.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Hourly pattern analysis saved to: hourly_demand_patterns.png")

In [ ]:
# Analyze daily patterns (day of week)
fig, axes = plt.subplots(2, 3, figsize=(20, 10))
fig.suptitle('Weekly Charging Demand Patterns by City', fontsize=16, fontweight='bold')

day_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

for idx, city in enumerate(CITIES):
    row = idx // 3
    col = idx % 3
    ax = axes[row, col]
    
    volume = volume_data[city].copy()
    
    # Extract day of week
    volume['day_of_week'] = volume['timestamp'].dt.dayofweek
    
    # Sum across all sites
    site_cols = [col for col in volume.columns if col not in ['timestamp', 'day_of_week']]
    volume['total_demand'] = volume[site_cols].sum(axis=1)
    
    # Average by day of week
    daily_avg = volume.groupby('day_of_week')['total_demand'].mean()
    
    # Plot
    ax.bar(range(7), daily_avg.values, color=sns.color_palette('husl', 7))
    ax.set_title(CITY_NAMES[city], fontweight='bold')
    ax.set_xlabel('Day of Week')
    ax.set_ylabel('Average Charging Sessions')
    ax.set_xticks(range(7))
    ax.set_xticklabels(day_names, rotation=45)
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('weekly_demand_patterns.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Weekly pattern analysis saved to: weekly_demand_patterns.png")

## 6. Clustering Analysis

Perform clustering analysis to identify patterns in charging site behavior for each city.

In [ ]:
# Prepare features for clustering
def prepare_clustering_features(city_code):
    """
    Prepare feature matrix for clustering analysis
    Features: total_volume, total_duration, avg_power, charger_num, area
    """
    sites = sites_data[city_code].copy()
    
    # Select relevant features
    feature_cols = ['total_volume', 'total_duration', 'avg_power', 'charger_num', 'area']
    
    # Create feature matrix
    X = sites[feature_cols].fillna(0)
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    return X_scaled, sites, feature_cols

print("Clustering preparation function defined")

In [ ]:
# Perform K-Means clustering for each city
clustering_results = {}
n_clusters = 4  # Number of clusters

fig, axes = plt.subplots(2, 3, figsize=(20, 12))
fig.suptitle('K-Means Clustering of Charging Sites (n=4 clusters)', fontsize=16, fontweight='bold')

for idx, city in enumerate(CITIES):
    row = idx // 3
    col = idx % 3
    ax = axes[row, col]
    
    # Prepare data
    X_scaled, sites, feature_cols = prepare_clustering_features(city)
    
    # Apply K-Means
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    clusters = kmeans.fit_predict(X_scaled)
    
    # Store results
    sites['cluster'] = clusters
    clustering_results[city] = sites
    
    # Apply PCA for visualization
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_scaled)
    
    # Plot clusters
    scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], 
                        c=clusters, cmap='viridis', 
                        s=100, alpha=0.6, edgecolors='black')
    
    # Plot cluster centers
    centers_pca = pca.transform(kmeans.cluster_centers_)
    ax.scatter(centers_pca[:, 0], centers_pca[:, 1], 
              c='red', marker='X', s=300, edgecolors='black', linewidths=2,
              label='Centroids')
    
    ax.set_title(f"{CITY_NAMES[city]}\n{len(sites)} sites", fontweight='bold')
    ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} var)')
    ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} var)')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('kmeans_clustering_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ K-Means clustering analysis saved to: kmeans_clustering_analysis.png")

In [ ]:
# Analyze cluster characteristics
print("\n📊 CLUSTER CHARACTERISTICS BY CITY\n")
print("="*80)

for city in CITIES:
    print(f"\n{CITY_NAMES[city]} ({city})")
    print("-" * 80)
    
    sites_clustered = clustering_results[city]
    
    for cluster_id in range(n_clusters):
        cluster_sites = sites_clustered[sites_clustered['cluster'] == cluster_id]
        
        print(f"\n  Cluster {cluster_id}: {len(cluster_sites)} sites")
        print(f"    Avg Volume: {cluster_sites['total_volume'].mean():,.0f}")
        print(f"    Avg Duration: {cluster_sites['total_duration'].mean():.1f} min")
        print(f"    Avg Power: {cluster_sites['avg_power'].mean():.1f} kW")
        print(f"    Avg Chargers: {cluster_sites['charger_num'].mean():.1f}")

In [ ]:
# Create cluster maps for each city
print("\nCreating cluster visualization maps...\n")

for city in CITIES:
    sites_clustered = clustering_results[city]
    
    # Calculate center
    center_lat = sites_clustered['latitude'].mean()
    center_lon = sites_clustered['longitude'].mean()
    
    # Create map
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=11,
        tiles='OpenStreetMap'
    )
    
    # Color scheme for clusters
    cluster_colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred']
    
    # Add markers for each cluster
    for cluster_id in range(n_clusters):
        cluster_sites = sites_clustered[sites_clustered['cluster'] == cluster_id]
        feature_group = folium.FeatureGroup(name=f'Cluster {cluster_id}')
        
        for _, site in cluster_sites.iterrows():
            folium.CircleMarker(
                location=[site['latitude'], site['longitude']],
                radius=6,
                popup=f"Cluster {cluster_id}<br>Site {site['site']}<br>Volume: {site['total_volume']:.0f}<br>Power: {site['avg_power']:.1f} kW",
                color=cluster_colors[cluster_id],
                fill=True,
                fillColor=cluster_colors[cluster_id],
                fillOpacity=0.7
            ).add_to(feature_group)
        
        feature_group.add_to(m)
    
    # Add layer control
    folium.LayerControl().add_to(m)
    
    # Save map
    output_path = f'{city.lower()}_cluster_map.html'
    m.save(output_path)
    print(f"✓ {CITY_NAMES[city]} cluster map saved to: {output_path}")

print("\nAll cluster maps created!")

## 7. Correlation Analysis

Analyze correlations between different charging metrics.

In [ ]:
# Create correlation heatmaps
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
fig.suptitle('Feature Correlation Analysis by City', fontsize=16, fontweight='bold')

for idx, city in enumerate(CITIES):
    row = idx // 3
    col = idx % 3
    ax = axes[row, col]
    
    sites = sites_data[city]
    
    # Select numeric features
    feature_cols = ['total_volume', 'total_duration', 'avg_power', 'charger_num', 'area', 'perimeter']
    corr_matrix = sites[feature_cols].corr()
    
    # Create heatmap
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0, square=True, ax=ax, cbar_kws={'shrink': 0.8})
    ax.set_title(CITY_NAMES[city], fontweight='bold')

plt.tight_layout()
plt.savefig('correlation_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Correlation analysis saved to: correlation_analysis.png")

## 8. Site Utilization Distribution

Analyze how charging volume is distributed across sites.

In [ ]:
# Analyze volume distribution
fig, axes = plt.subplots(2, 3, figsize=(20, 10))
fig.suptitle('Charging Volume Distribution Across Sites', fontsize=16, fontweight='bold')

for idx, city in enumerate(CITIES):
    row = idx // 3
    col = idx % 3
    ax = axes[row, col]
    
    sites = sites_data[city]
    volumes = sites['total_volume'].values
    
    # Create histogram
    ax.hist(volumes, bins=30, edgecolor='black', alpha=0.7)
    ax.axvline(volumes.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {volumes.mean():.0f}')
    ax.axvline(np.median(volumes), color='green', linestyle='--', linewidth=2, label=f'Median: {np.median(volumes):.0f}')
    
    ax.set_title(f"{CITY_NAMES[city]}", fontweight='bold')
    ax.set_xlabel('Total Charging Sessions')
    ax.set_ylabel('Number of Sites')
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('volume_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Volume distribution analysis saved to: volume_distribution.png")

## 9. Summary Report

Generate a comprehensive summary of key insights.

In [ ]:
print("\n" + "="*80)
print(" " * 20 + "CHARGED DATASET ANALYSIS SUMMARY")
print("="*80)

print("\n📍 GEOGRAPHIC COVERAGE")
print("-" * 80)
for city in CITIES:
    sites = sites_data[city]
    print(f"  {CITY_NAMES[city]:15s}: {len(sites):3d} sites, "
          f"{sites['charger_num'].sum():4.0f} chargers")

print("\n⚡ CHARGING ACTIVITY")
print("-" * 80)
total_sessions = 0
for city in CITIES:
    sites = sites_data[city]
    sessions = sites['total_volume'].sum()
    total_sessions += sessions
    print(f"  {CITY_NAMES[city]:15s}: {sessions:,.0f} sessions")
print(f"\n  {'TOTAL':15s}: {total_sessions:,.0f} charging sessions across all cities")

print("\n📊 KEY INSIGHTS")
print("-" * 80)

# Find city with highest average usage
avg_usage = {city: sites_data[city]['total_volume'].mean() for city in CITIES}
max_usage_city = max(avg_usage, key=avg_usage.get)
print(f"  • Highest avg usage per site: {CITY_NAMES[max_usage_city]} ({avg_usage[max_usage_city]:,.0f} sessions/site)")

# Find city with highest power
avg_power = {city: sites_data[city]['avg_power'].mean() for city in CITIES}
max_power_city = max(avg_power, key=avg_power.get)
print(f"  • Highest avg charging power: {CITY_NAMES[max_power_city]} ({avg_power[max_power_city]:.1f} kW)")

# Find city with longest sessions
avg_duration = {city: sites_data[city]['total_duration'].mean() for city in CITIES}
max_duration_city = max(avg_duration, key=avg_duration.get)
print(f"  • Longest avg session duration: {CITY_NAMES[max_duration_city]} ({avg_duration[max_duration_city]:.1f} minutes)")

print("\n📁 OUTPUTS GENERATED")
print("-" * 80)
print("  Maps:")
print("    • all_cities_overview_map.html - Global overview of all cities")
for city in CITIES:
    print(f"    • {city.lower()}_charging_time_map.html - Time-based heatmap for {CITY_NAMES[city]}")
    print(f"    • {city.lower()}_cluster_map.html - Cluster analysis for {CITY_NAMES[city]}")

print("\n  Charts:")
print("    • city_comparison_overview.png - Infrastructure comparison")
print("    • hourly_demand_patterns.png - Hourly usage patterns")
print("    • weekly_demand_patterns.png - Weekly usage patterns")
print("    • kmeans_clustering_analysis.png - Clustering visualization")
print("    • correlation_analysis.png - Feature correlations")
print("    • volume_distribution.png - Volume distribution analysis")

print("\n  Data:")
print("    • city_summary_statistics.csv - Summary statistics")

print("\n" + "="*80)
print(" " * 25 + "ANALYSIS COMPLETE!")
print("="*80 + "\n")

## 10. Interactive Exploration Tips

### How to Use the Time-Based Maps:
1. Open any `*_charging_time_map.html` file in your web browser
2. Use the time slider at the bottom to scrub through different time periods
3. Click the play button to see an animated view of charging demand evolution
4. Toggle layers on/off using the layer control in the top-right corner

### How to Use the Cluster Maps:
1. Open any `*_cluster_map.html` file in your browser
2. Use the layer control to show/hide specific clusters
3. Click on markers to see detailed site information
4. Each cluster represents sites with similar charging characteristics

### Next Steps for Analysis:
- Investigate why certain clusters form (geographic, demographic, or infrastructure factors)
- Compare peak hours across cities to understand cultural/timezone differences
- Correlate weather data with charging patterns
- Build predictive models for future demand